# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [130]:
#Imports 

import pandas as pd
import numpy as np

#Data 
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10910 entries, 0 to 10909
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     10910 non-null  int64  
 1   Customer                       10910 non-null  object 
 2   State                          10279 non-null  object 
 3   Customer Lifetime Value        10910 non-null  float64
 4   Response                       10279 non-null  object 
 5   Coverage                       10910 non-null  object 
 6   Education                      10910 non-null  object 
 7   Effective To Date              10910 non-null  object 
 8   EmploymentStatus               10910 non-null  object 
 9   Gender                         10910 non-null  object 
 10  Income                         10910 non-null  int64  
 11  Location Code                  10910 non-null  object 
 12  Marital Status                 10910 non-null 

In [31]:
#1

df_new = df[(df["Total Claim Amount"]>= 1000) & (df["Response"] == "Yes")]
df_new.describe()

,Unnamed: 0,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Total Claim Amount
count,67.000000,67.000000,67.000000,67.000000,64.000000,67.000000,64.000000,67.000000,67.000000
mean,5376.985075,11502.854777,36568.671642,213.268657,17.000000,54.492537,0.531250,2.268657,1176.368262
std,3462.135444,6370.018320,22786.320061,56.452603,8.417668,31.895365,0.689346,2.489871,138.315227
min,189.000000,3508.569533,0.000000,98.000000,4.000000,5.000000,0.000000,1.000000,1008.000000
25%,1832.500000,7840.165778,16181.000000,181.000000,8.000000,24.000000,0.000000,1.000000,1027.200000
50%,5331.000000,10179.717040,46503.000000,214.000000,19.000000,43.000000,0.000000,1.000000,1176.278800
75%,8507.500000,13736.132500,55687.000000,271.000000,22.000000,89.000000,1.000000,2.000000,1300.800000
max,10708.000000,25807.063000,71210.000000,283.000000,33.000000,96.000000,2.000000,8.000000,1358.400000


In [66]:
#2
df_new2 = df[(df["Response"] == "Yes")].pivot_table(index="Policy Type", columns="Gender", values="Total Claim Amount", aggfunc='mean')
df_new2

Gender,F,M
Policy Type,,
Corporate Auto,433.738499,408.582459
Personal Auto,452.965929,457.010178
Special Auto,453.280164,429.527942


In [120]:
#3
df_new3 = df.groupby("State")
for state,group in df_new3:
    if int(group.count()[1]) >= 500:
        print(f"State:{state} has more than 500 Costumers")
        


State:Arizona has more than 500 Costumers
State:California has more than 500 Costumers
State:Nevada has more than 500 Costumers
State:Oregon has more than 500 Costumers
State:Washington has more than 500 Costumers


/tmp/ipykernel_4746/2679368029.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if int(group.count()[1]) >= 500:
/tmp/ipykernel_4746/2679368029.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if int(group.count()[1]) >= 500:
/tmp/ipykernel_4746/2679368029.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if int(group.count()[1]) >= 500:
/tmp/ipykernel_4746/2679368029.py:4: FutureWarning: Series.__getitem__ treating keys as 

In [122]:
#4
df.pivot_table(index="Education", columns="Gender", values="Customer Lifetime Value", aggfunc='mean')

Gender,F,M
Education,,
Bachelor,7874.269478,7703.601675
College,7748.823325,8052.459288
Doctor,7328.508916,7415.333638
High School or Below,8675.220201,8149.687783
Master,8157.053154,8168.832659


In [124]:
df.pivot_table(index="Education", columns="Gender", values="Customer Lifetime Value", aggfunc='min')

Gender,F,M
Education,,
Bachelor,1904.000852,1898.007675
College,1898.683686,1918.119700
Doctor,2395.570000,2267.604038
High School or Below,2144.921535,1940.981221
Master,2417.777032,2272.307310


In [126]:
df.pivot_table(index="Education", columns="Gender", values="Customer Lifetime Value", aggfunc='max')

Gender,F,M
Education,,
Bachelor,73225.95652,67907.27050
College,61850.18803,61134.68307
Doctor,44856.11397,32677.34284
High School or Below,55277.44589,83325.38119
Master,51016.06704,50568.25912


,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,10905,FE99816,Nevada,15563.369440,No,Premium,Bachelor,1/19/11,Unemployed,F,...,NaN,7,Personal Auto,Personal L1,Offer3,Web,1214.400000,Luxury Car,Medsize,A
10906,10906,KX53892,Oregon,5259.444853,No,Basic,College,1/6/11,Employed,F,...,0.0,6,Personal Auto,Personal L3,Offer2,Branch,273.018929,Four-Door Car,Medsize,A
10907,10907,TL39050,Arizona,23893.304100,No,Extended,Bachelor,2/6/11,Employed,F,...,0.0,2,Corporate Auto,Corporate L3,Offer1,Web,381.306996,Luxury SUV,Medsize,NaN
10908,10908,WA60547,California,11971.977650,No,Premium,College,2/13/11,Employed,F,...,4.0,6,Personal Auto,Personal L1,Offer1,Branch,618.288849,SUV,Medsize,A
